In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split

# Paths to the train and test directories
train_dir = r"D:\Data Science\Data Frames\Data Slayer 2024\data-slayer-2-0-machine-learning-competition\train"
test_dir = r"D:\Data Science\Data Frames\Data Slayer 2024\data-slayer-2-0-machine-learning-competition\test"

# Constants
IMG_SIZE = 224  # Resize all images to 224x224
BATCH_SIZE = 32


In [2]:
# Function to preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))  # Resize
    img = img_to_array(img) / 255.0  # Normalize to [0,1]
    return img

# Load training data
train_data = []
train_labels = []

for subject in os.listdir(train_dir):
    subject_path = os.path.join(train_dir, subject)
    for category in os.listdir(subject_path):
        category_path = os.path.join(subject_path, category)
        label = 1 if category == 'fall' else 0  # Label fall = 1, non-fall = 0
        for subcategory in os.listdir(category_path):
            subcategory_path = os.path.join(category_path, subcategory)
            if os.path.isdir(subcategory_path):  # If subcategory is a folder
                for img_file in os.listdir(subcategory_path):
                    img_path = os.path.join(subcategory_path, img_file)
                    train_data.append(preprocess_image(img_path))
                    train_labels.append(label)
            else:  # If file directly
                train_data.append(preprocess_image(subcategory_path))
                train_labels.append(label)

# Convert to NumPy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")


Training samples: 3435, Validation samples: 859


In [3]:
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def extract_pose_features(image):
    """
    Extracts human pose features from an image using MediaPipe Pose.
    Returns a flat array of keypoint coordinates and visibility.
    """
    results = pose.process(image)
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        # Flatten [x, y, z, visibility] for each keypoint into a single list
        features = []
        for lm in landmarks:
            features.extend([lm.x, lm.y, lm.z, lm.visibility])
        return features
    else:
        # If no keypoints detected, return a zero vector of appropriate length
        return [0] * (33 * 4)  # 33 keypoints with x, y, z, visibility

# Extract pose features for all training images
pose_features = []
for img in train_data:
    # Reshape the image back to its original size for pose estimation
    reshaped_img = np.uint8(img * 255.0)  # Scale back to [0, 255]
    features = extract_pose_features(reshaped_img)
    pose_features.append(features)

# Convert pose features to a NumPy array
pose_features = np.array(pose_features)
print(f"Pose features shape: {pose_features.shape}")

# Combine pose features with labels
X_pose_train, X_pose_val, y_pose_train, y_pose_val = train_test_split(
    pose_features, train_labels, test_size=0.2, random_state=42
)

print(f"Pose Training samples: {len(X_pose_train)}, Validation samples: {len(X_pose_val)}")


Pose features shape: (4294, 132)
Pose Training samples: 3435, Validation samples: 859


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Define a fully connected neural network
model = Sequential([
    Dense(128, activation='relu', input_dim=132),  # 33 keypoints x 4 features (x, y, z, visibility)
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(X_pose_train, y_pose_train, validation_data=(X_pose_val, y_pose_val), epochs=20, batch_size=32)


c:\Users\L E G I O N\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        17,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,289 (110.50 KB)

 Trainable params: 27,841 (108.75 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7591 - loss: 0.5217 - val_accuracy: 0.8009 - val_loss: 0.4848
Epoch 2/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8474 - loss: 0.3739 - val_accuracy: 0.8685 - val_loss: 0.3478
Epoch 3/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8680 - loss: 0.3257 - val_accuracy: 0.8289 - val_loss: 0.3817
Epoch 4/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8673 - loss: 0.3316 - val_accuracy: 0.8778 - val_loss: 0.3123
Epoch 5/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8852 - loss: 0.2855 - val_accuracy: 0.8568 - val_loss: 0.3291
Epoch 6/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8745 - loss: 0.2931 - val_accuracy: 0.8766 - val_loss: 0.3006
Epoch 7/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8909 - loss: 0.2775 - val_accuracy: 0.7939 - val_loss: 0.4389
Epoch 8/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8761 - loss: 0.2943 - val_accuracy: 0.

In [5]:
# Prepare test data
test_images = []
test_image_paths = []

# Read all images from the test directory
for img_file in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_file)
    test_image_paths.append(img_path)
    img = preprocess_image(img_path)  # Resize and normalize
    test_images.append(img)

# Convert to NumPy array
test_images = np.array(test_images)

# Extract pose features for the test dataset
test_pose_features = []
for img in test_images:
    reshaped_img = np.uint8(img * 255.0)  # Scale back to [0, 255]
    features = extract_pose_features(reshaped_img)
    test_pose_features.append(features)

# Convert to NumPy array
test_pose_features = np.array(test_pose_features)
print(f"Test pose features shape: {test_pose_features.shape}")


Test pose features shape: (2152, 132)


In [8]:
# Predict using the trained model
test_predictions = model.predict(test_pose_features)
test_predictions_binary = (test_predictions > 0.5).astype(int)  # Convert probabilities to binary (0 or 1)

# Map predictions to test image filenames
results = pd.DataFrame({
    "id": [os.path.basename(path) for path in test_image_paths],
    "label": test_predictions_binary.flatten()
})

# Display the results
print(results.head())


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step
               id  label
0  00396960ae.jpg      0
1  004773d4e8.jpg      0
2  009b266d98.jpg      1
3  009ed56c89.jpg      0
4  00a16cb3ae.jpg      0


In [9]:
# Save predictions to a CSV file
output_path = "test_predictions.csv"
results.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")


Predictions saved to test_predictions.csv
